In [1]:
%load_ext autoreload 
%autoreload 2

from imports import *
import queue_system2 as qs
from queue_system import *

seed = random.seed(10)
seed_np = np.random.seed(10)

Dictionary representation:
0: [(4, 500.0), (5, 200.0), (6, 50.0), (9, 50.0), (7, 500.0), (13, 500.0), (15, 50.0), (12, 50.0), (10, 500.0), (11, 200.0), (8, 200.0), (14, 200.0)]
1: [(7, 500.0), (9, 50.0), (13, 500.0), (15, 50.0), (8, 200.0), (14, 200.0)]
2: [(10, 500.0), (12, 50.0), (13, 500.0), (15, 50.0), (11, 200.0), (14, 200.0)]
3: [(13, 500.0), (15, 50.0), (14, 200.0)]
4: [(0, 500.0)]
5: [(0, 200.0)]
6: [(0, 50.0)]
7: [(1, 500.0), (0, 500.0)]
8: [(1, 200.0), (0, 200.0)]
9: [(1, 50.0), (0, 50.0)]
10: [(2, 500.0), (0, 500.0)]
11: [(0, 200.0), (2, 200.0)]
12: [(2, 50.0), (0, 50.0)]
13: [(3, 500.0), (0, 500.0), (1, 500.0), (2, 500.0)]
14: [(0, 200.0), (1, 200.0), (2, 200.0), (3, 200.0)]
15: [(3, 50.0), (0, 50.0), (1, 50.0), (2, 50.0)]


# Dynamic Programming

## Compute $V^{q_\star} $

In [2]:
%%time

P, r = compute_transitions_and_rewards_experts()

# Initialize the value function
V_agg = np.zeros(num_states)

P, r = compute_transitions_and_rewards_experts()

# Perform value iteration
num_iterations = 1000
tolerance = 1e-5

optimal_weights = np.zeros(num_states, dtype=int)

for i in range(num_iterations):
    delta = 0
    for s in range(num_states):
        v = V_agg[s]
        max_q_value = -np.inf
        best_expert = None
        for a in range(num_exp):
            q_value = 0
            for prob, next_s, reward in zip(P[s, a], list(range(num_states)), r[s, a]):
                q_value += prob * (reward + discount * V_agg[next_s])
            if q_value > max_q_value:
                max_q_value = q_value
                best_expert = a
        V_agg[s] = max_q_value
        optimal_weights[s] = best_expert
        #print(best_expert)
        delta = max(delta, abs(v - V_agg[s]))
    
    print('delta -> ', delta)
    if delta < tolerance:
        break

# Extract the optimal policy

delta ->  1083.9475319925361
delta ->  199.38374094720166
delta ->  96.58400903404838
delta ->  54.39841806576203
delta ->  30.487089454614136
delta ->  17.316092397173577
delta ->  9.94777402392873
delta ->  5.851176304839555
delta ->  4.005170173097426
delta ->  2.753137761944302
delta ->  1.8838655149872636
delta ->  1.2847520284474854
delta ->  0.8739331413373748
delta ->  0.5932938370562795
delta ->  0.4021421803287666
delta ->  0.27223747884794136
delta ->  0.18411223571746405
delta ->  0.12441371351906128
delta ->  0.08401778028036233
delta ->  0.05670799898784651
delta ->  0.03825867040409037
delta ->  0.025802504530275883
delta ->  0.017396737163011267
delta ->  0.011726543930990374
delta ->  0.007902905248101888
delta ->  0.005325163085586837
delta ->  0.003587737443396577
delta ->  0.0024169074584676764
delta ->  0.0016280182574774926
delta ->  0.0010965418310036057
delta ->  0.0007385220024787031
delta ->  0.0004973688944858168
delta ->  0.0003349458472854394
delta ->  0.00

In [3]:
with open('values/V_agg2.pkl', 'wb') as output:
    pickle.dump(V_agg, output) 

## Compute $V^\star $

In [4]:
%%time

# Initialize the value function


P, r = compute_transitions_and_rewards_actions()

# Perform value iteration
num_iterations = 1000
tolerance = 1e-5

policy = np.zeros(num_states, dtype=int)
V_star = np.zeros(num_states)

for i in range(num_iterations):
    delta = 0
    for s in range(num_states):
        v = V_star[s]
        #print(v)
        max_q_value = -np.inf
        best_expert = None
        for a in range(num_actions):
            if np.sum(P[s, a, :]) != 0:
                q_value = 0
                for prob, next_s, reward in zip(P[s, a], list(range(num_states)), r[s, a]):
                    q_value += prob * (reward + discount * V_star[next_s])
                if q_value > max_q_value:
                    max_q_value = q_value
                    best_expert = a
        V_star[s] = max_q_value
        #print(s, max_q_value, delta, v)
        policy[s] = best_expert
        delta = max(delta, abs(v - V_star[s]))
    
    print('delta -> ', delta)
    if delta < tolerance:
        break

delta ->  759.7834877860779
delta ->  199.93489458094965
delta ->  96.71979418005148
delta ->  54.652733913435185
delta ->  30.566306057649456
delta ->  17.393353319775542
delta ->  10.038206882758288
delta ->  5.98917728310596
delta ->  4.058163200488025
delta ->  2.7903340653776496
delta ->  1.9097748680645878
delta ->  1.3026711824024488
delta ->  0.8862606323757092
delta ->  0.6017452986554161
delta ->  0.40792385506638595
delta ->  0.2761868136131085
delta ->  0.1868072498142226
delta ->  0.12625153981984738
delta ->  0.08527042220660519
delta ->  0.05756145553220904
delta ->  0.0388399769480543
delta ->  0.026198348410161998
delta ->  0.017666235679399733
delta ->  0.011909993351409298
delta ->  0.008027762540208983
delta ->  0.00541013131007162
delta ->  0.0036455536757955542
delta ->  0.002456244144639186
delta ->  0.0016547793333074878
delta ->  0.0011147459416918082
delta ->  0.0007509041839313113
delta ->  0.0005057903646417117
delta ->  0.00034067305105622836
delta ->  0.00

In [5]:
P, r = compute_transitions_and_rewards_actions()

Dictionary representation:
0: [(4, 500.0), (5, 200.0), (6, 50.0), (9, 50.0), (7, 500.0), (13, 500.0), (15, 50.0), (12, 50.0), (10, 500.0), (11, 200.0), (8, 200.0), (14, 200.0)]
1: [(7, 500.0), (9, 50.0), (13, 500.0), (15, 50.0), (8, 200.0), (14, 200.0)]
2: [(10, 500.0), (12, 50.0), (13, 500.0), (15, 50.0), (11, 200.0), (14, 200.0)]
3: [(13, 500.0), (15, 50.0), (14, 200.0)]
4: [(0, 500.0)]
5: [(0, 200.0)]
6: [(0, 50.0)]
7: [(1, 500.0), (0, 500.0)]
8: [(1, 200.0), (0, 200.0)]
9: [(1, 50.0), (0, 50.0)]
10: [(2, 500.0), (0, 500.0)]
11: [(0, 200.0), (2, 200.0)]
12: [(2, 50.0), (0, 50.0)]
13: [(3, 500.0), (0, 500.0), (1, 500.0), (2, 500.0)]
14: [(0, 200.0), (1, 200.0), (2, 200.0), (3, 200.0)]
15: [(3, 50.0), (0, 50.0), (1, 50.0), (2, 50.0)]


In [6]:
with open('values/V_star2.pkl', 'wb') as output:
    pickle.dump(V_star, output)      

In [7]:
V_exp = []
for i in range(num_exp):
    w = np.zeros(num_exp)
    w[i] = 1
    weights_exp = np.array([w]* num_states)
    P_avg, r_avg = compute_transitions_and_rewards_avg(weights_exp)
    V_exp.append(compute_value_bellman(P_avg, r_avg))

weights_dict_opt = np.zeros((num_states, num_exp))

for s in state_space_ind:
    w = np.zeros(num_exp)
    i = optimal_weights[int(s)]
    w[i] = 1
    weights_dict_opt[int(s)] = w
    
P_opt, r_opt = compute_transitions_and_rewards_avg(weights_dict_opt)
V_opt = compute_value_bellman(P_opt, r_opt)

In [8]:
with open('values/V_opt2.pkl', 'wb') as output:
    pickle.dump(V_opt, output)  
with open('values/V_exp2.pkl', 'wb') as output:
    pickle.dump(V_exp, output)  